In [1]:
pip install progress

Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
import pandas as pd
import openpyxl
import numpy as np
from progress.bar import Bar

In [2]:
data = pd.read_csv('SNK1.csv', delimiter=';', names=['slovo','tag'])
tags = open('tags.json')
tags_data = json.load(tags)

In [3]:
data

,slovo,tag
0,Kde,PD
1,tá,PFfs1
2,ľudská,AAfs1x
3,duša,SSfs1
4,drieme,VKesc+
...,...,...
116641,na,Eu6
116642,Zemi,SSfs6:r
116643,veľmi,Dx
116644,premenlivé,AAns1x


In [4]:
data = data.dropna()

In [23]:
data

,slovo,tag
0,Kde,PD
1,tá,PFfs1
2,ľudská,AAfs1x
3,duša,SSfs1
4,drieme,VKesc+
...,...,...
116641,na,Eu6
116642,Zemi,SSfs6:r
116643,veľmi,Dx
116644,premenlivé,AAns1x


In [19]:
df = pd.DataFrame(columns=['Slovo', 'Slovný druh', 'Rod', 'Číslo', 'Pád', 'Paradigma', 'Stupeň', 'Aglutinovanosť', 'Slovesná forma', 'Vid', 'Osoba', 'Negácia','Druh','Forma','Spojenie s pádom', 'Kondicionálnosť'])

In [20]:
for i,word in data.iterrows():
    if word.tag[0:3] =='id=':
        row = {}
        row['Slovo'] = '"'
        row['Slovný druh'] = 'Interpunkcia'
    else:
        row = {}
        row['Slovo'] = word.slovo
        for i,letter in enumerate(word.tag):
            if i == 0:
                row['Slovný druh'] = tags_data[letter]['slovakName']
            else:
                if bool(tags_data[word.tag[0]]['tags']):
                    if letter in tags_data[word.tag[0]]['tags']:
                        row[tags_data[word.tag[0]]['tags'][letter]['label']] = tags_data[word.tag[0]]['tags'][letter]['value']
                    else:
                        if "r" == word.tag[i]:
                            row['Informácia'] = 'Príslovka'
                            break
                        if "?" == word.tag[i]:
                            row['Informácia'] = 'Chybný zápis'
                            break
                        if ":" == word.tag[i]:
                            if ":r" == word.tag[i:i+2]:
                                row['Informácia'] = 'Vlastné meno'
                                break
                            else:
                                row['Informácia'] = 'Chybný zápis'
                                break
    df = df.append(row, ignore_index = True)

In [22]:
df

,Slovo,Slovný druh,Rod,Číslo,Pád,Paradigma,Stupeň,Aglutinovanosť,Slovesná forma,Vid,Osoba,Negácia,Druh,Forma,Spojenie s pádom,Kondicionálnosť,stupeň,Informácia
0,Kde,Zámeno,NaN,NaN,NaN,príslovková,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tá,Zámeno,ženský,jednotné,nominatív,zmiešaná,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ľudská,Prídavné meno,ženský,jednotné,nominatív,adjektívna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pozitív,NaN
3,duša,Podstatné meno,ženský,jednotné,nominatív,substantívna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,drieme,Sloveso,NaN,jednotné,NaN,NaN,NaN,NaN,indikatív,nedokonavý,tretia,afirmácia,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106026,na,Predložka,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nevokalizovaná,lokál,NaN,NaN,NaN
106027,Zemi,Podstatné meno,ženský,jednotné,lokál,substantívna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vlastné meno
106028,veľmi,Príslovka,NaN,NaN,NaN,NaN,pozitív,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106029,premenlivé,Prídavné meno,stredný,jednotné,nominatív,adjektívna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pozitív,NaN


In [24]:
df.to_csv('SNK_anotacia.csv', sep=';',encoding='utf-8-sig',index=False)